# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2f6cf7ab20>
├── 'a' --> tensor([[-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986]])
└── 'x' --> <FastTreeValue 0x7f2f6cf7ae20>
    └── 'c' --> tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                        [-1.7054,  0.1086, -0.0537,  0.3634],
                        [ 0.0320, -0.6255,  0.5534,  0.0112]])

In [4]:
t.a

tensor([[-0.0260,  1.4482,  0.5199],
        [ 0.4596,  3.5255,  0.8986]])

In [5]:
%timeit t.a

77.1 ns ± 0.0799 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2f6cf7ab20>
├── 'a' --> tensor([[-0.0934,  1.2316, -0.2734],
│                   [ 0.4018,  0.2758,  1.1074]])
└── 'x' --> <FastTreeValue 0x7f2f6cf7ae20>
    └── 'c' --> tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                        [-1.7054,  0.1086, -0.0537,  0.3634],
                        [ 0.0320, -0.6255,  0.5534,  0.0112]])

In [7]:
%timeit t.a = new_value

77.5 ns ± 0.0841 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0260,  1.4482,  0.5199],
               [ 0.4596,  3.5255,  0.8986]]),
    x: Batch(
           c: tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112]]),
       ),
)

In [10]:
b.a

tensor([[-0.0260,  1.4482,  0.5199],
        [ 0.4596,  3.5255,  0.8986]])

In [11]:
%timeit b.a

65 ns ± 0.0679 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4437, -0.2711, -1.0411],
               [ 0.6869, -0.0642, -0.0135]]),
    x: Batch(
           c: tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112]]),
       ),
)

In [13]:
%timeit b.a = new_value

545 ns ± 0.151 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

892 ns ± 4.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 21.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

147 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

149 µs ± 859 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2e92775b50>
├── 'a' --> tensor([[[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]],
│           
│                   [[-0.0260,  1.4482,  0.5199],
│                    [ 0.4596,  3.5255,  0.8986]]])
└── 'x' --> <FastTreeValue 0x7f2e92775af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 32.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2e927756d0>
├── 'a' --> tensor([[-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986],
│                   [-0.0260,  1.4482,  0.5199],
│                   [ 0.4596,  3.5255,  0.8986]])
└── 'x' --> <FastTreeValue 0x7f2ea53286d0>
    └── 'c' --> tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                        [-1.7054,  0.1086, -0.0537,  0.3634],
                 

In [23]:
%timeit t_cat(trees)

31.2 µs ± 40.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 59.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3088,  0.8533,  0.5704, -0.7271],
                       [-1.7054,  0.1086, -0.0537,  0.3634],
                       [ 0.0320, -0.6255,  0.5534,  0.0112]],
              
                      [[ 0.3088,  0.8533,  0.5704, -0.7271],
                       [-1.7054,  0.1086, -0.0537,  0.3634],
                       [ 0.0320, -0.6255,  0.5534,  0.0112]],
              
                      [[ 0.3088,  0.8533,  0.5704, -0.7271],
                       [-1.7054,  0.1086, -0.0537,  0.3634],
                       [ 0.0320, -0.6255,  0.5534,  0.0112]],
              
                      [[ 0.3088,  0.8533,  0.5704, -0.7271],
                       [-1.7054,  0.1086, -0.0537,  0.3634],
                       [ 0.0320, -0.6255,  0.5534,  0.0112]],
              
                      [[ 0.3088,  0.8533,  0.5704, -0.7271],
                       [-1.7054,  0.1086, -0.0537,  0.3634],
                       [ 0.0320, -0.6255,  0.5534,  0.0112]],

In [26]:
%timeit Batch.stack(batches)

85.2 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112],
                      [ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112],
                      [ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112],
                      [ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112],
                      [ 0.3088,  0.8533,  0.5704, -0.7271],
                      [-1.7054,  0.1086, -0.0537,  0.3634],
                      [ 0.0320, -0.6255,  0.5534,  0.0112],
                      [ 0.3088,  0.8533,  0.5704, -0.7271],
                   

In [28]:
%timeit Batch.cat(batches)

155 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

350 µs ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
